In [1]:
using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks

g = readgraph(:hepth);
a = adjacency_list(g::AGraph);
N = size(a)[1];

A0 = sparse(1:N,1:N,0*vec(ones(Int64,N,1)),N,N);
for i=1:N
    l = a[i];
    for j=1:length(l)
        A0[i,l[j]]=1;
        A0[l[j],i]=1;
    end
end
A,p = largest_component(A0);


In [2]:
d = sum(A,2);
s = sum(d);


r = 100;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[17.9408, 12.9756, 12.7694, 12.0805, 10.6007, 10.5225, 10.1741, 9.92351, 9.93585, 9.59244, 9.43578, 9.27954, 9.14447, 9.02801, 8.78118, 8.8017, 8.71793, 8.57075, 8.49993, 8.3321, 8.31025, 8.12706, 8.11336, 8.06621, 8.01476, 7.98814, 7.9008, 7.83198, 7.77316, 7.71999, 7.63748, 7.6447, 7.50683, 7.51068, 7.48383, 7.37389, 7.41587, 7.37408, 7.34812, 7.31366, 7.2268, 7.25441, 7.12943, 7.11456, 7.15796, 7.10785, 7.0531, 7.04601, 6.9962, 7.04111, 6.90969, 6.90694, 6.95144, 6.88343, 6.8542, 6.8821, 6.7968, 6.81763, 6.74199, 6.74685, 6.71419, 6.66964, 6.67772, 6.6348, 6.64612, 6.64666, 6.57123, 6.5906, 6.4733, 6.54566, 6.46795, 6.44685, 6.36551, 6.38877, 6.3976, 6.32118, 6.31443, 6.24261, 6.20713, 6.24645, 6.21623, 6.19783, 6.14857, 6.16629, 6.15704, 6.14238, 6.08306, 6.11156, 6.09927, 6.07102, 6.04708, 6.01593, 5.97755, 5.95567, 5.92634, 5.95492, 5.9322, 5.94144, 5.91587, 5.89394]

In [19]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:95]';

n_clusters = 200;
n_it_Kmeans = 1000;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;
# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 2
Number of communities: 81
Modularity: 0.7233455219553038


In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:100
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 0
Number of communities: 26
Modularity: 0.5139623736388335
n: 10
